# Investigation of Open Resolvers

- make resolve the test domains from the adnet study
- assess AD, STATUS and pseudo data from resolver

#### Reference Data Format
```
{
  "time_epoch": "1633644148866",
  "time_human": "2021-10-07T22:02:28",
  "ip_server": "141.12.174.43",
  "ip_client": "181.174.106.42",
  "request_method": "",
  "protocol": "",
  "host_header": "",
  "server_alias": "",
  "port_server": "",
  "url_path": "",
  "filename": "",
  "query": "?test=session-finish&tok=23377425&time=1633644148441",
  "time_served_ms": "",
  "status": "",
  "errlog_reqest_id": "",
  "user_agent": ""
}
```

In [1]:
import numpy as np
import dns.message, dns.query, dns.rdataclass, dns.rdatatype, dns.flags, dns.exception, dns.name
from tqdm import tqdm
import concurrent
import logging
import pandas as pd
import csv

IN = dns.rdataclass.from_text("IN")
NS = dns.rdatatype.from_text("NS")
SOA = dns.rdatatype.from_text("SOA")
DS = dns.rdatatype.from_text("DS")
A = dns.rdatatype.from_text("A")
AAAA = dns.rdatatype.from_text("AAAA")
RRSIG = dns.rdatatype.from_text("RRSIG")

NUM_CONCURRENTS = 25
REPO_DIR = '../../dnssec-downgrade-data/'
DATA_DIR = REPO_DIR + '/2021-10-08_open-resolvers-anon/'  # location of input/raw and processed data

JSON_FILE_OUT = DATA_DIR + "/downg-open-anon.json"
PICKLE_FILE_OUT = DATA_DIR + "/downg-open-anon.pickle.gz"

In [2]:

STUDY_DOMAIN = "downgrade.dedyn.io"
STUDY_DOMAIN_OLD = "resolver-downgrade-attack.dedyn.io"

DOWNGRADE_VULNERABILITY_TESTS = [
    # Downgrade Vulnerabilities
    "mitm-ra-ds8-ds13.ds13-ds16-dnskey13-dnskey16",
    "mitm-ra-ds8-ds13.ds8-ds15-dnskey15",
    "mitm-ra-ds8-ds13-ds15.ds13-ds16-dnskey13-dnskey16",
    "mitm-ra-ds8-ds13-ds15.ds16",
    "mitm-ra-ds8-ds13-ds15.ds8-ds15-dnskey15",
    "mitm-ra-ds8-ds13-ds15-ds16.ds8-ds16-dnskey16",
    "mitm-ra-ds8-ds13-ds15-ds16.ds16",
    "mitm-ra-ds8-ds13-ds15-ds16.ds8-ds15-dnskey15",
    "mitm-rs13-ra.ds8-ds16-dnskey16",
    "mitm-rs13-ra.ds8-ds13-dnskey8",
    "mitm-rs13-ra.ds8-ds15-dnskey8-dnskey15",
    "mitm-rs13-ra.ds13-ds16-dnskey13-dnskey16",
    "mitm-rs13-ra.ds16",
    "mitm-rs15-ra.ds8-ds16-dnskey16",
    "mitm-rs15-ra.ds8-ds13-dnskey13",
    "mitm-rs15-ra.ds8-dnskey8",
    "mitm-rs15-ra.ds8-ds16-dnskey8-dnskey16",
    "mitm-rs15-ra.ds13-ds16-dnskey13-dnskey16",
    "mitm-rs15-ra.ds16",
    "mitm-rs15-ra.ds8-ds16-dnskey16",
    "mitm-rs15-ra.ds8-ds15-dnskey15",
    "mitm-rs16-ra.ds8-ds13-dnskey13",
    "mitm-rs16-ra.ds15-ds16-dnskey15",
    "mitm-rs16-ra.ds13-ds16-dnskey16",
    "mitm-rs16-ra.ds13-ds16-dnskey13-dnskey16",
    "mitm-rs16-ra.ds8-ds13-dnskey8",
    "mitm-rs16-ra.ds8-ds15-dnskey8-dnskey15",
    "mitm-rs16-ra.ds13-dnskey13",
    "mitm-rs16-ra.ds8-ds13-dnskey8-dnskey13",
    "mitm-rs16-ra.ds13-ds15-dnskey15",
    "mitm-rs8-ra.ds8-ds16-dnskey16",
    "mitm-rs8-ra.ds15-ds16-dnskey16",
    "mitm-rs8-ra.ds13-ds16-dnskey16",
    "mitm-rs8-ra.ds13-ds16-dnskey13-dnskey16",
    "mitm-rs8-ra.ds13-dnskey13",
    "mitm-rs8-ra.ds13-ds15-dnskey13-dnskey15",
    "mitm-rs8-ra.ds8-ds13-dnskey8-dnskey13",
    "mitm-rs8-ra.ds8-dnskey8",
    "mitm-rs8-ra.ds16",
    "mitm-rs8-ra.ds8-ds16-dnskey8-dnskey16",
]

ALGORITHM_SUPPORT_TESTS = [
    "mitm-ra.ds5-dnskey5",
    "mitm-ra.ds8-dnskey8",
    "mitm-ra.ds10-dnskey10",
    "mitm-ra.ds13-dnskey13",
    "mitm-ra.ds14-dnskey14",
    "mitm-ra.ds15-dnskey15",
    "mitm-ra.ds16-dnskey16",
]

TEST_NAMES = {
     *DOWNGRADE_VULNERABILITY_TESTS,
     *ALGORITHM_SUPPORT_TESTS,
    
     # Housekeeping
     "broken",  # Proxy for DNSSEC validation
     "session-finish",  # substitute for empty child / parent domain
}


DEV_RESOLVERS = ['8.8.8.8', '127.1.3.5']
RESOLVERS_FILE = DATA_DIR + "/anon-resolvers.txt"
RESOLVERS_COUNT = 15

DNSSEC_CHECK_DOMAIN = "broken." + STUDY_DOMAIN_OLD  # must return SERVFAIL
RESOLUTION_CHECK_DOMAIN = "ns.x.dnsstu.de"
RESOLUTION_CHECK_ADDRESS = "141.12.174.24"
QUERY_DOMAINS = [
    *['.'.join((d, STUDY_DOMAIN)) for d in DOWNGRADE_VULNERABILITY_TESTS + ALGORITHM_SUPPORT_TESTS]
]

In [3]:
RESOLVERS = []
with open(RESOLVERS_FILE, 'r') as rfp:
    for line in rfp:
        RESOLVERS.append(line.strip())
len(RESOLVERS)
RESOLVERS = RESOLVERS[:RESOLVERS_COUNT]
RESOLVERS

['198.144.171.15',
 '1.20.253.124',
 '218.17.26.2',
 '112.53.176.77',
 '60.29.205.180',
 '202.88.43.79',
 '113.121.78.17',
 '103.123.250.83',
 '110.44.229.146',
 '124.227.119.109',
 '62.169.203.65',
 '180.123.89.32',
 '1.32.63.76',
 '189.76.81.150',
 '201.16.253.69']

### Resolver Check Routine

In [4]:
def query(qname, resolver, rdtype=A):
    q = dns.message.make_query(qname, A, want_dnssec=True)
    try:
        return dns.query.udp(q, where=resolver, timeout=2)
    except dns.exception.Timeout:
        return dns.query.udp(q, where=resolver, timeout=5)


def is_non_resolver_response(response):
    # With blackjack. And nice ladies...
    if response is None or response.rcode() != 0 or len(response.answer) == 0:
        return True
    for ans_rrset in response.answer:
        try:
            for ans_r in ans_rrset:
                if ans_r.address == RESOLUTION_CHECK_ADDRESS:
                    return False
        except AttributeError:
            pass
    return True
    

def check_resolver(resolver):
    resolver_result = dict()
    resolver_result['token'] = resolver
    # "session-finish", "broken", algo support tests, downgrade vulnerability tests
    caught_timeout = False
    try:
        # 1. check for resolution at all
        try:
            response = query(RESOLUTION_CHECK_DOMAIN, resolver, A)
        except dns.exception.Timeout:
            caught_timeout = True
        if caught_timeout or is_non_resolver_response(response):
            logging.warning(f"Not a resolver: {resolver}")
            return {
                **resolver_result,
                **{test_name: False for test_name in TEST_NAMES},
                "session-finish": False
            }

        # 2. check test domains
        for test_name in DOWNGRADE_VULNERABILITY_TESTS + ALGORITHM_SUPPORT_TESTS:
            test_domain = ".".join((test_name, STUDY_DOMAIN))
            resolution_success = False
            caught_timeout = False
            try:
                response = query(test_domain, resolver, A)
            except dns.exception.Timeout:
                caught_timeout = True
            resolution_success = (not caught_timeout) and response.rcode() == 0
            resolver_result[test_name] = resolution_success

        # 3. check dnssec validation domain (don't skip on failure)
        # broken.resolver-downgrade-attack.dedyn.io
        caught_timeout = False
        try:
            response = query("broken.resolver-downgrade-attack.dedyn.io", resolver, A)
        except dns.exception.Timeout:
            caught_timeout = True
        resolution_success = (not caught_timeout) and response.rcode() == 0
        resolver_result["broken"] = resolution_success

        # 4. add pseudo session finish domain
        resolver_result["session-finish"] = True
        return resolver_result
    except Exception as e:
        logging.warning(f"caught exception ass {e}")
        e.rname = rname
        e.r = resolver
        e.d = d
        raise

In [5]:
executor = concurrent.futures.ThreadPoolExecutor(NUM_CONCURRENTS)

def run_queries(resolvers):
    errors = []
    results = []
    
#     for r in resolvers:
#         try:
#             results.append()
    
    futures = [executor.submit(check_resolver, r) for r in resolvers]
    with tqdm(total=len(futures), desc="Querying") as pbar:
        for future in concurrent.futures.as_completed(futures):
            pbar.update(1)
            if future.exception():
                logging.warning(f"{future.exception().r}: {future.exception()}")
                errors.append(future.exception().r)
            else:
                results.append(future.result())
    return results, errors

In [6]:
results, errors = run_queries(DEV_RESOLVERS)
results

Querying: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:47<00:00, 23.90s/it]


[{'token': '127.1.3.5',
  'mitm-rs15-ra.ds8-ds13-dnskey13': False,
  'mitm-ra.ds13-dnskey13': False,
  'mitm-ra.ds8-dnskey8': False,
  'mitm-ra-ds8-ds13.ds13-ds16-dnskey13-dnskey16': False,
  'mitm-rs16-ra.ds13-ds16-dnskey16': False,
  'mitm-ra-ds8-ds13-ds15.ds16': False,
  'mitm-rs15-ra.ds13-ds16-dnskey13-dnskey16': False,
  'mitm-rs8-ra.ds15-ds16-dnskey16': False,
  'mitm-rs16-ra.ds8-ds13-dnskey8': False,
  'mitm-rs15-ra.ds8-ds16-dnskey8-dnskey16': False,
  'mitm-rs16-ra.ds8-ds13-dnskey13': False,
  'mitm-ra.ds10-dnskey10': False,
  'mitm-rs15-ra.ds16': False,
  'mitm-rs8-ra.ds8-ds16-dnskey8-dnskey16': False,
  'session-finish': False,
  'mitm-ra.ds14-dnskey14': False,
  'mitm-ra.ds16-dnskey16': False,
  'mitm-rs16-ra.ds13-dnskey13': False,
  'mitm-rs8-ra.ds13-dnskey13': False,
  'mitm-ra-ds8-ds13-ds15-ds16.ds8-ds16-dnskey16': False,
  'mitm-rs15-ra.ds8-dnskey8': False,
  'mitm-ra-ds8-ds13-ds15-ds16.ds8-ds15-dnskey15': False,
  'mitm-rs13-ra.ds8-ds13-dnskey8': False,
  'mitm-rs8-ra.d

In [8]:

df_results = pd.DataFrame(results)
df_results.set_index('token', inplace=True)
df_results.to_pickle(PICKLE_FILE_OUT, compression='gzip')
df_results

,mitm-rs15-ra.ds8-ds13-dnskey13,mitm-ra.ds13-dnskey13,mitm-ra.ds8-dnskey8,mitm-ra-ds8-ds13.ds13-ds16-dnskey13-dnskey16,mitm-rs16-ra.ds13-ds16-dnskey16,mitm-ra-ds8-ds13-ds15.ds16,mitm-rs15-ra.ds13-ds16-dnskey13-dnskey16,mitm-rs8-ra.ds15-ds16-dnskey16,mitm-rs16-ra.ds8-ds13-dnskey8,mitm-rs15-ra.ds8-ds16-dnskey8-dnskey16,...,mitm-ra.ds15-dnskey15,mitm-rs13-ra.ds8-ds16-dnskey16,mitm-rs16-ra.ds8-ds15-dnskey8-dnskey15,broken,mitm-rs8-ra.ds13-ds15-dnskey13-dnskey15,mitm-rs15-ra.ds8-ds16-dnskey16,mitm-rs16-ra.ds15-ds16-dnskey15,mitm-rs8-ra.ds13-ds16-dnskey16,mitm-rs13-ra.ds16,mitm-ra-ds8-ds13.ds8-ds15-dnskey15
token,,,,,,,,,,,,,,,,,,,,,
127.1.3.5,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8.8.8.8,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
